In [12]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from scipy.sparse import csr_matrix
from sklearn.metrics import accuracy_score
import torch
import os
import seaborn as sns
from torch.utils.data import DataLoader,TensorDataset
import pickle
import json
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
from sae.preprocess import get_meta_info,get_freq_emmbeddings, get_tf_idfs_emmbeddings,get_bert_embeddings
from sae.models import DEEPmodel_2
import time
import joblib
%autoreload 2

# import data

In [14]:
mode = "summary"

In [15]:
data = pd.read_csv('ratings_formatted.csv')
data

,Titre,rating,summary,comment,utilite_num,auteur_note,auteur_nb,genre_note,genre_nb,editor_note,editor_nb
0,Its Only Art If Its Well Hung!,4.0,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,1.000000,4.0000,1.0,4.335691,13149.0,2.500000,0.0
1,Dr. Seuss: American Icon,5.0,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,1.000000,4.6000,10.0,4.258935,107780.0,4.212478,2885.0
2,Dr. Seuss: American Icon,5.0,Essential for every personal and Public Library,"If people become the books they read and if ""t...",0.909091,4.6000,10.0,4.258935,107780.0,4.212478,2885.0
3,Dr. Seuss: American Icon,4.0,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",1.000000,4.6000,10.0,4.258935,107780.0,4.212478,2885.0
4,Dr. Seuss: American Icon,4.0,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,1.000000,4.6000,10.0,4.258935,107780.0,4.212478,2885.0
...,...,...,...,...,...,...,...,...,...,...,...
2999580,The Idea of History,4.0,Difficult,"This is an extremely difficult book to digest,...",0.736842,4.3125,16.0,4.232240,89980.0,4.272727,11.0
2999581,The Idea of History,4.0,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...,1.000000,4.3125,16.0,4.232240,89980.0,4.272727,11.0
2999582,The Idea of History,4.0,Easier reads of those not well versed in histo...,"This is a good book but very esoteric. ""What i...",0.000000,4.3125,16.0,4.232240,89980.0,4.272727,11.0
2999583,The Idea of History,5.0,"Yes, it is cheaper than the University Bookstore","My daughter, a freshman at Indiana University,...",0.090909,4.3125,16.0,4.232240,89980.0,4.272727,11.0


In [16]:
def sentiment(data): 
    Y_valid_sentiment = data['rating'].values.astype(int)
    Y_valid_sentiment[Y_valid_sentiment < 3] = 0
    Y_valid_sentiment[Y_valid_sentiment == 3] = 1
    Y_valid_sentiment[Y_valid_sentiment > 3] = 2
    return(Y_valid_sentiment)

In [17]:
data=data.head(80000)
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)
data_train['sentiment']=sentiment(data_train)
data_test['sentiment']=sentiment(data_test)
data_train

,Titre,rating,summary,comment,utilite_num,auteur_note,auteur_nb,genre_note,genre_nb,editor_note,editor_nb,sentiment
3822,"The Story Of Civilization (Eleven Volumes, Com...",1.0,misleading item description.,My order was for a complete set which was supp...,0.000000,4.656250,32.0,4.232240,89980.0,4.188958,118090.0,0
35562,Witch of Blackbird Pond (Bookwise Guide),3.0,The Witch,I read this about 10 years ago while still in ...,1.000000,4.264033,481.0,4.398051,1334.0,2.500000,0.0,1
4883,The Weed and the Winter Solstice,5.0,Another masterpiece from the South.,This revised edition of THE WEED &amp; THE WIN...,0.333333,5.000000,2.0,4.113632,824273.0,4.030864,162.0,2
71170,A Connecticut Yankee in King Arthur's Court (O...,5.0,This book is an onion...more and more layers,I read this book when I was young and going th...,1.000000,4.109030,8383.0,4.113632,824273.0,3.994839,1744.0,2
25665,On heroes and hero-worship and the heroic in h...,5.0,Intellectual reading,I am on an intellectual quest to understand wh...,0.000000,4.087866,239.0,4.182338,20270.0,4.309940,2978.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
6265,Tess of the D'Urbervilles: A pure woman (Harpe...,4.0,Tess,The classic tale of Tess of the D'Urbervilles ...,1.000000,4.122254,6691.0,2.500000,0.0,2.500000,0.0,2
54886,Do-Gooders: How Liberals Hurt Those They Claim...,5.0,A body check to the liberal solar plexus,"Ahhh, there's nothing better than seeing a new...",0.733333,2.500000,0.0,4.164932,6724.0,2.500000,0.0,2
76820,Estonian Textbook: Grammar Exercises Conversat...,4.0,Definitely not the best language course ever f...,"Unlike most of the previous reviews, I don't s...",0.500000,4.666667,6.0,4.230047,14810.0,4.243590,78.0,2
860,Cruel and Unusual (G K Hall Large Print Book S...,5.0,Excellent!,Excellent Book! Action packed! Good suspense! ...,0.000000,3.263437,1321.0,4.113632,824273.0,4.167773,1055.0,2


In [18]:
x_meta_train=data_train[["auteur_note","auteur_nb",'genre_note','genre_nb','editor_note',"editor_nb"]].values
x_meta_test=data_test[["auteur_note","auteur_nb",'genre_note','genre_nb','editor_note',"editor_nb"]].values
yTrain=data_train[['rating']].values
yTest=data_test[['rating']].values
x_meta_train

array([[4.65625000e+00, 3.20000000e+01, 4.23224050e+00, 8.99800000e+04,
        4.18895757e+00, 1.18090000e+05],
       [4.26403326e+00, 4.81000000e+02, 4.39805097e+00, 1.33400000e+03,
        2.50000000e+00, 0.00000000e+00],
       [5.00000000e+00, 2.00000000e+00, 4.11363226e+00, 8.24273000e+05,
        4.03086420e+00, 1.62000000e+02],
       ...,
       [4.66666667e+00, 6.00000000e+00, 4.23004727e+00, 1.48100000e+04,
        4.24358974e+00, 7.80000000e+01],
       [3.26343679e+00, 1.32100000e+03, 4.11363226e+00, 8.24273000e+05,
        4.16777251e+00, 1.05500000e+03],
       [4.54334183e+00, 1.01980000e+04, 4.11363226e+00, 8.24273000e+05,
        4.44155844e+00, 5.15900000e+03]])

In [19]:
yTrain

array([[1.],
       [3.],
       [5.],
       ...,
       [4.],
       [5.],
       [5.]])

# Importe embeding model

## Mots simple

In [20]:
Freqmodel = joblib.load('freq_model.pkl')
x_emmbed_freq_train,_ = get_freq_emmbeddings(list(data_train[mode].values),model=Freqmodel)
x_emmbed_freq_test,_ = get_freq_emmbeddings(list(data_test[mode].values),model=Freqmodel)
x_emmbed_freq_train.shape

(64000, 1000)

In [21]:
xtrain_freq = np.concatenate((x_meta_train,x_emmbed_freq_train),axis=1)
xtest_freq = np.concatenate((x_meta_test,x_emmbed_freq_test),axis=1)
print(f"xtrain shape: {xtrain_freq.shape}")
print(f"xtest shape: {xtest_freq.shape}")
print(f"ytrain shape: {yTrain.shape}")
print(f"ytest shape: {yTest.shape}")

xtrain shape: (64000, 1006)
xtest shape: (16000, 1006)
ytrain shape: (64000, 1)
ytest shape: (16000, 1)


In [22]:
nb_model = GradientBoostingClassifier(random_state=42)
nb_model.fit(xtrain_freq, yTrain)

c:\Users\natha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(random_state=42)

In [23]:
y_pred_nb = nb_model.predict(xtest_freq)
accuracy_nb = accuracy_score(yTest, y_pred_nb)
f1_nb = f1_score(yTest, y_pred_nb, average='weighted')
print(f"Gradient Boosting:")
print(f"Accuracy: {accuracy_nb:.4f}")
print(f"F1-score: {f1_nb:.4f}\n")

Gradient Boosting:
Accuracy: 0.6313
F1-score: 0.5375



In [24]:
joblib.dump(nb_model, 'Re_freq_gradientboosting.pkl')

['Re_freq_gradientboosting.pkl']

## Tf idf

In [25]:
idfmodel = joblib.load('tfidf_model.pkl')
x_emmbed_idf_train,inutile = get_tf_idfs_emmbeddings(list(data_train[mode].values),model=idfmodel)
x_emmbed_idf_test,inutile = get_tf_idfs_emmbeddings(list(data_test[mode].values),model=idfmodel)

In [26]:
xtrain_idf = np.concatenate((x_meta_train,x_emmbed_idf_train),axis=1)
xtest_idf = np.concatenate((x_meta_test,x_emmbed_idf_test),axis=1)
print(f"xtrain shape: {xtrain_idf.shape}")
print(f"xtest shape: {xtest_idf.shape}")
print(f"ytrain shape: {yTrain.shape}")
print(f"ytest shape: {yTest.shape}")

xtrain shape: (64000, 1006)
xtest shape: (16000, 1006)
ytrain shape: (64000, 1)
ytest shape: (16000, 1)


In [27]:
nb_model = GradientBoostingClassifier(random_state=42)
nb_model.fit(xtrain_idf, yTrain)

c:\Users\natha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(random_state=42)

In [28]:
y_pred_nb = nb_model.predict(xtest_idf)
accuracy_nb = accuracy_score(yTest, y_pred_nb)
f1_nb = f1_score(yTest, y_pred_nb, average='weighted')
print(f"Gradient Boosting:")
print(f"Accuracy: {accuracy_nb:.4f}")
print(f"F1-score: {f1_nb:.4f}\n")

Gradient Boosting:
Accuracy: 0.6296
F1-score: 0.5355



In [29]:
joblib.dump(nb_model, 'RE_idf_gradientboosting.pkl')

['RE_idf_gradientboosting.pkl']

In [9]:
sub = data_train.head(512)

## bert A Faire tournée

In [10]:
x_emmbed_bert_train =  get_bert_embeddings(list(data_train[mode].values),batch_size = 256).numpy()
x_emmbed_bert_test =  get_bert_embeddings(list(data_test[mode].values),batch_size = 256).numpy()

**************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************ZERO

*************************************************************************************************************************************************************


In [11]:
xtrain_bert = np.concatenate((x_meta_train,x_emmbed_bert_train),axis=1)
xtest_bert= np.concatenate((x_meta_test,x_emmbed_bert_test),axis=1)
print(f"xtrain shape: {xtrain_bert.shape}")
print(f"xtest shape: {xtest_bert.shape}")
print(f"ytrain shape: {yTrain.shape}")
print(f"ytest shape: {yTest.shape}")

xtrain shape: (160000, 774)
xtest shape: (40000, 774)
ytrain shape: (160000, 1)
ytest shape: (40000, 1)


In [12]:
nb_model = GradientBoostingClassifier(random_state=42)
nb_model.fit(xtrain_bert, yTrain)

c:\Users\natha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(random_state=42)

In [13]:
y_pred_nb = nb_model.predict(xtest_bert)
accuracy_nb = accuracy_score(yTest, y_pred_nb)
f1_nb = f1_score(yTest, y_pred_nb, average='weighted')
print(f"Gradient Boosting:")
print(f"Accuracy: {accuracy_nb:.4f}")
print(f"F1-score: {f1_nb:.4f}\n")

Gradient Boosting:
Accuracy: 0.8366
F1-score: 0.7998



In [ ]:
#joblib.dump(nb_model, 'bert_gradientboosting.pkl')

['bert_gradientboosting.pkl']